In [42]:
import pandas as pd
import numpy as np

# 假定已经加载了日线数据到daily_data DataFrame
daily_data = pd.read_csv("SS2401_daily.csv")

# 设置K1和K2参数
K1 = 0.4
K2 = 0.6
N = 3
# 假设daily_data是日线DataFrame
daily_data['HH'] = daily_data['high'].rolling(window=N, min_periods=1).max()
daily_data['LC'] = daily_data['close'].rolling(window=N, min_periods=1).min()
daily_data['HC'] = daily_data['close'].rolling(window=N, min_periods=1).max()
daily_data['LL'] = daily_data['low'].rolling(window=N, min_periods=1).min()

daily_data['Range'] = daily_data.apply(lambda x: max(x['HH'] - x['LC'], x['HC'] - x['LL']), axis=1)
daily_data['upperLine'] = daily_data['open'] + K1 * daily_data['Range']
daily_data['lowerLine'] = daily_data['open'] - K2 * daily_data['Range']



# 为每天的交易准备好Range数据
daily_range_data = daily_data[['trade_date', 'upperLine', 'lowerLine']].copy()

# 加载数据
data = pd.read_csv("future_taobao_ss2401_tick.csv")

# 假设trade_date的格式是YYYYMMDD
daily_range_data['trade_date'] = pd.to_datetime(daily_range_data['trade_date'], format='%Y%m%d')

# 直接使用 datetime 而非转换为 date
data['date'] = pd.to_datetime(data['datetime']).dt.normalize()

# 由于daily_range_data['trade_date']已经是datetime类型，可以直接合并
data = pd.merge(data, daily_range_data, left_on='date', right_on='trade_date', how='left')

print(daily_range_data)
# print(data)



# 设置平仓触发条件，这里假设为价格相对于开仓价变动2%
profit_trigger = 0.008

# 初始化仓位和资金
position = {'type': None, 'entry_price': 0, 'quantity': 0}
capital = 100000
transaction_log = []
last_trade_date = None  # 记录最后交易的日期

# 模拟交易，添加日内交易限制和及时止损逻辑
for index, row in data.iterrows():
    current_date = row['date']
    # print(row[['last_price','upperLine','lowerLine']])
    if position['type'] is None:  # 检查是否已有仓位
        # 检查是否已在当天进行过交易
        if last_trade_date == current_date:
            continue  # 如果是，则跳过此次循环，不执行交易
        if row['last_price'] > row['upperLine']:  # 开多仓条件
            last_trade_date = current_date  # 更新最后交易日期
            position['type'] = 'long'
            position['entry_price'] = row['last_price']
            position['quantity'] = capital / row['last_price']  # 假设全部资金投入
            capital -= position['quantity'] * row['last_price']  # 扣除投入资金
            transaction_log.append({'datetime': row['datetime'], 'action': 'OPEN LONG', 'price': row['last_price'], 'quantity': position['quantity']})
        elif row['last_price'] < row['lowerLine']:  # 开空仓条件
            last_trade_date = current_date  # 更新最后交易日期
            position['type'] = 'short'
            position['entry_price'] = row['last_price']
            position['quantity'] = capital / row['last_price']  # 假设全部资金投入
            transaction_log.append({'datetime': row['datetime'], 'action': 'OPEN SHORT', 'price': row['last_price'], 'quantity': position['quantity']})
    # 日内交易逻辑：确保在每日结束前平仓
    elif position['type'] is not None and (index == len(data) - 1 or data.at[index + 1, 'date'] != row['date']):
        # 如果当前持有仓位且即将到达日末或数据集末尾，则强制平仓
        if position['type'] == 'long':
            capital += position['quantity'] * row['last_price']
            profit = position['quantity'] * (row['last_price'] - position['entry_price'])
            transaction_log.append({'datetime': row['datetime'], 'action': 'FORCE CLOSE LONG', 'price': row['last_price'], 'profit': profit})
        elif position['type'] == 'short':
            profit = position['quantity'] * (position['entry_price'] - row['last_price'])
            capital += profit
            transaction_log.append({'datetime': row['datetime'], 'action': 'FORCE CLOSE SHORT', 'price': row['last_price'], 'profit': profit})
        position = {'type': None, 'entry_price': 0, 'quantity': 0}
        continue
    else:  # 已有仓位，检查平仓条件
        if position['type'] == 'long':
            if row['last_price'] >= position['entry_price'] * (1 + profit_trigger) or row['last_price'] <= position['entry_price'] * (1 - profit_trigger):
                capital += position['quantity'] * row['last_price']
                profit = position['quantity'] * (row['last_price'] - position['entry_price'])
                transaction_log.append({'datetime': row['datetime'], 'action': 'CLOSE LONG', 'price': row['last_price'], 'profit': profit})
                position = {'type': None, 'entry_price': 0, 'quantity': 0}
        elif position['type'] == 'short':
            if row['last_price'] <= position['entry_price'] * (1 - profit_trigger) or row['last_price'] >= position['entry_price'] * (1 + profit_trigger):
                profit = position['quantity'] * (position['entry_price'] - row['last_price'])
                capital += profit  # 直接加上盈利
                transaction_log.append({'datetime': row['datetime'], 'action': 'CLOSE SHORT', 'price': row['last_price'], 'profit': profit})
                position = {'type': None, 'entry_price': 0, 'quantity': 0}

# 输出交易记录和最终资金
transaction_df = pd.DataFrame(transaction_log)
transaction_df.to_csv('transaction_log')

# 从交易日志DataFrame中计算胜率和盈亏比
# 假设profit列已经正确地记录了每次交易的盈亏

# 筛选出盈利和亏损的交易
profitable_trades = transaction_df[transaction_df['profit'] > 0]
losing_trades = transaction_df[transaction_df['profit'] < 0]

# 计算胜率
win_rate = len(profitable_trades) / len(transaction_df) if len(transaction_df) > 0 else 0

# 计算盈亏比
average_profit = profitable_trades['profit'].mean() if not profitable_trades.empty else 0
average_loss = losing_trades['profit'].mean() if not losing_trades.empty else 0
profit_loss_ratio = abs(average_profit / average_loss) if average_loss != 0 else 0


print(f"Final capital: {capital}")
print(f"Win Rate: {win_rate:.2f}")
print(f"Profit/Loss Ratio: {profit_loss_ratio:.2f}")





    trade_date  upperLine  lowerLine
0   2023-12-29    13930.0    13605.0
1   2023-12-28    14075.0    13525.0
2   2023-12-27    14002.0    13397.0
3   2023-12-26    14007.0    13827.0
4   2023-12-25    13851.0    13611.0
..         ...        ...        ...
227 2023-01-30    16754.0    16519.0
228 2023-01-20    16694.0    16459.0
229 2023-01-19    16573.0    16403.0
230 2023-01-18    16576.0    16411.0
231 2023-01-17        NaN        NaN

[232 rows x 3 columns]
Final capital: 91537.2342928161
Win Rate: 0.18
Profit/Loss Ratio: 0.64
